# Matrix and vector Computation

## Pure Python version


In [35]:
def run_experiment(experiment, iterations, label, baseline=None):
    try:
        t = experiment.run_experiment(nruns)
        extra_runs = max(int(30 / t) - 1, 2)
        for i in xrange(extra_runs):
            test = experiment.run_experiment(nruns)
            t = min(t, test)
    except Exception as e:
        print "Could not run: %s: %s" % (label, e)
        raise

    speedup_label = ''
    if baseline:
        speedup_label = "[%0.2fx speedup]" % (baseline / t)
    _format = (label, t, t / float(nruns), speedup_label)
    print "%s: %0.2fs (%es per iteration)%s" % _format
    return t, (baseline or 0) / t

nruns = 50

In [40]:
from __future__ import division
import time

grid_size = (1024, )

class diffusion_python():
    
    @staticmethod
    def laplacian(grid):
        lap = [0.0, ] * grid_size[0]
        xmax, = grid_size
        for i in xrange(grid_size[0]):
            grid_xx = grid[(i + 1) % xmax] + grid[(i - 1) % xmax] - 2.0 * grid[i]
            lap[i] = grid_xx
        return lap

    @staticmethod
    def evolve(grid, dt, D=1.0):
        lap = laplacian(grid)
        for i in xrange(grid_size[0]):
            grid[i] += D * lap[i] * dt
        return grid

    @staticmethod
    def run_experiment(num_iterations):
        # setting up initial conditions
        grid = [0.0, ] * grid_size[0]

        block_low = int(grid_size[0] * .4)
        block_high = int(grid_size[0] * .5)
        for i in xrange(block_low, block_high):
            grid[i] = 0.005

        start = time.time()
        for i in range(num_iterations):
            grid = evolve(grid, 0.1)
        return time.time() - start


In [41]:
run_experiment(diffusion_python, nruns, "Pure Python")

Pure Python: 0.03s (5.367422e-04s per iteration)


(0.02683711051940918, 0.0)

In [27]:
from numpy import (roll, zeros)
import time

grid_shape = (1024, 1024)


class diffusion_numpy:
    @staticmethod
    def laplacian(grid):
        return roll(grid, +1, 0) + roll(grid, -1, 0) + \
            roll(grid, +1, 1) + roll(grid, -1, 1) - 4 * grid

    @staticmethod
    def evolve(grid, dt, D=1):
        return grid + dt * D * laplacian(grid)

    @staticmethod
    def run_experiment(num_iterations):
        grid = zeros(grid_shape)

        block_low = int(grid_shape[0] * .4)
        block_high = int(grid_shape[0] * .5)
        grid[block_low:block_high, block_low:block_high] = 0.005

        start = time.time()
        for i in range(num_iterations):
            grid = evolve(grid, 0.1)
        return time.time() - start
    

In [28]:
run_experiment(diffusion_numpy, nruns, "numpy")

numpy: 0.62s (1.246354e-02s per iteration)


(0.6231770515441895, 0.0)